<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/ml_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Импорт**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
from IPython.display import SVG, display, HTML

In [3]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Mounted at ./content


In [4]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [5]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  sample_data
.file-revisions-by-id
.shortcut-targets-by-id
MyDrive
.Trash


In [6]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [7]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv

--2021-05-22 22:01:00--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-22 22:01:00 ERROR 404: Not Found.

--2021-05-22 22:01:00--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1192227 (1.1M) [text/plain]
Saving to: ‘new_anns60k.csv’

new_anns60k.csv     100%[===================>]   1.14M  --.-KB/s    in 0.07s   

2021

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [8]:
# with open('coref_df_anns60k.csv') as csvfile:
with open('new_anns60k.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter='\t')
  with open('new_df_anns60k.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    c = 0
    for row in reader:
      new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
      if len(new_row) > 9:
        new_row = new_row[:9]
      writer.writerow(new_row)


In [9]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('new_df_anns60k.csv', delimiter='\t')

Проверим данные

In [10]:
anns_df

,TOKEN,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,IS_ANSWER,HEAD,NER,COREFERENCE_CLUSTER
0,обсуждаем,1.0,0.0,0.0,0.0,0.0,обсуждаем,0.0,0.0
1,препарат,1.0,0.0,0.0,1.0,0.0,обсуждаем,0.0,0.0
2,рисперидон,1.0,0.0,0.0,2.0,0.0,препарат,1.0,2.0
3,рисполепт,1.0,0.0,0.0,3.0,0.0,рисперидон,1.0,3.0
4,сперидан,1.0,0.0,0.0,4.0,0.0,рисполепт,1.0,4.0
...,...,...,...,...,...,...,...,...,...
15405,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15406,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15407,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15408,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [11]:
import numpy as np

In [12]:
cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k.csv', delimiter=' ')
cor_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
1,препарат,[-0.17725945 0.94834274 0.90867615 0.850609...,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
2,рисперидон,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,[-0.17725945 0.94834274 0.90867615 0.850609...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,0
3,рисполепт,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,[ 0.15259643 1.0673474 0.84192747 -0.077515...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,0
4,сперидан,[ 0.08634768 0.893356 0.55127096 -0.065075...,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.02878256 0.29778534 0.18375699 -0.021691...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,[ 0.5048941 0.81449467 0.29763442 0.603937...,0,22,68,0,7,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26493,таможенные,[ 0.73242044 -0.24206167 -0.6556247 -0.476650...,0,22,68,0,8,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26494,пошлины,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,0,22,68,0,9,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0
26495,как,[ 1.0349948e+00 7.3793733e-01 -3.8374668e-01 ...,0,22,68,0,10,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0


In [13]:
cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

0.8436395154681279

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [14]:
cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]
cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]
cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]
cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]

Добавим размеченные данные, и посмотрим, что получилось.

In [15]:
cor_df['NER'] = anns_df['NER']
cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
# cor_df

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [344]:
df = cor_df.iloc[:13000,:]

In [345]:
df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,"[-0.208233938, 0.996180058, -0.279894114, -0.9...",0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0.0,0.0
1,препарат,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[-0.17725945, 0.94834274, 0.90867615, 0.850609...",0.0,0.0
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
3,рисполепт,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.012426247, 1.0408522, 0.6913349, -0.1638528...",1.0,3.0
4,сперидан,"[0.08634768, 0.893356, 0.55127096, -0.06507572...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.02878256, 0.29778534, 0.18375699, -0.021691...",1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,hifu,"[0.5528928, 0.5487517, 0.02201277, 0.06551573,...",1,15,1,1,18,NaN,NaN,NaN,NaN,NaN,obj,проходил,NaN,NaN,masc,VERB,"[0.77081865, 0.8555614, 0.13789928, 0.19477502...","[0.0264964812, 0.00272976467, 0.00271321484, 0...","[0.6981856, 0.48279706, 0.16776918, -0.2164765...",0.0,0.0
12996,терапию,"[1.3447013, 0.8065633, -0.09939983, 0.08727561...",1,15,1,1,19,inan,accs,femn,NaN,NOUN,obj,проходил,NaN,NaN,masc,VERB,"[0.77081865, 0.8555614, 0.13789928, 0.19477502...","[0.0264964812, 0.00272976467, 0.00271321484, 0...","[0.6981856, 0.48279706, 0.16776918, -0.2164765...",0.0,0.0
12997,и,"[0.5048941, 0.81449467, 0.29763442, 0.60393727...",1,15,1,1,20,NaN,NaN,NaN,NaN,CONJ,cc,подскажет,NaN,NaN,NaN,VERB,"[0.40671584, 0.3654391, 0.4083926, -0.68571347...","[0.0264964812, 0.00272976467, 0.00271321484, 0...","[0.38592952, 1.0004519, 0.14486013, 0.24140544...",0.0,0.0
12998,подскажет,"[0.40671584, 0.3654391, 0.4083926, -0.68571347...",1,15,1,1,21,NaN,NaN,NaN,3per,VERB,conj,проходил,NaN,NaN,masc,VERB,"[0.77081865, 0.8555614, 0.13789928, 0.19477502...","[0.0264964812, 0.00272976467, 0.00271321484, 0...","[0.6981856, 0.48279706, 0.16776918, -0.2164765...",0.0,0.0


Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [346]:
len(df.loc[df['NER'] == 1])

513

Из них местоимений (прономинальные анафоры).

In [347]:
len(df.loc[df['NER'] == 3])

94

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [348]:
len(df.loc[df['NER'] == 2])

195

In [349]:
len(df.loc[df['NER'].isin([1,2,3])])

802

In [350]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [351]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
3,рисполепт,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.012426247, 1.0408522, 0.6913349, -0.1638528...",1.0,3.0
4,сперидан,"[0.08634768, 0.893356, 0.55127096, -0.06507572...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.02878256, 0.29778534, 0.18375699, -0.021691...",1.0,4.0
21,лекарством,"[0.598588, -0.07832722, 0.23088205, -0.3704158...",0,1,1,1,10,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,"[1.3479745, 0.34477097, 0.20343542, 0.10220611...","[0.05498165, -0.00549405, 0.01865345, -0.03142...","[0.42120528, 0.21546456, 0.00441866, -0.240917...",2.0,0.0
142,рисполепта,"[0.20826544, 0.5118653, 0.6703367, -0.22502856...",0,1,1,13,7,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,"[0.5131939, 0.9708054, 0.53676724, 0.09788364,...","[0.11585727, 0.1283627, 0.02299715, -0.0095005...","[0.28126505, 0.3790152, 0.5397224, -0.09368, -...",1.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12785,завершению,"[0.31051013, 0.10955906, 0.39861053, -0.069338...",0,15,0,16,1,inan,datv,neut,NaN,NOUN,obl,сделали,NaN,NaN,NaN,VERB,"[0.18231322, 0.7513995, -0.6068166, -0.1418591...","[0.02632562, 0.20721841, -0.03911034, 0.037120...","[0.13199103, 0.31303656, 0.09326021, 0.0468936...",1.0,263.0
12791,нашему,"[0.75860929, 0.86483002, -0.13026427, -0.15006...",0,15,0,17,2,NaN,datv,neut,NaN,ADJF,det,ужасу,inan,datv,masc,NOUN,"[0.901275, 0.16275163, -0.37415406, 0.3714117,...","[-0.02689476, 0.0346927, 0.03875368, 0.0688789...","[0.606111884, 0.528075039, 0.182860225, -0.179...",3.0,263.0
12854,к,"[0.4536144, 0.19132005, 0.4959847, -0.20833261...",0,15,0,22,3,NaN,NaN,NaN,NaN,PREP,case,абраксану,anim,accs,femn,NOUN,"[0.5982271, 1.0975795, 0.52676016, 0.0132821, ...","[0.09760628, 0.08554257, 0.02943124, -0.035544...","[0.4536144, 0.19132005, 0.4959847, -0.20833261...",1.0,332.0
12876,стадию,"[0.745928, 0.7292406, 0.04893015, 0.1760835, -...",0,15,0,25,7,inan,accs,femn,NaN,NOUN,obj,имеет,NaN,NaN,NaN,VERB,"[0.750769973, 0.43797487, 0.616638899, 0.30344...","[0.056758128, 0.039144114, 0.073397294, 0.0356...","[0.30142662, 0.52905554, -0.21734424, 0.204744...",2.0,0.0


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [352]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
576,зелдокс,"[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",0,2,0,0,2,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.03948287, 0.24082112, 0.22588146, -0.017884...","[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",1.0,2.0
1870,оланзапин,"[-0.082683861, 1.2017089, 0.67815673, -0.22599...",0,3,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01915067, 0.0875298, 0.09080387, 0.00378973...","[-0.082683861, 1.2017089, 0.67815673, -0.22599...",1.0,2.0
5555,галоперидол,"[-0.03019532, 1.2394575, 0.18222564, -0.215572...",0,4,0,0,2,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[-0.03747297, 0.5554195, 0.15133946, -0.011017...","[-0.10372739, 1.0939001, 0.5454509, 0.3175185,...",1.0,2.0
6200,тиаприд,"[0.27581334, 0.80696917, 1.0271364, -0.1037090...",0,5,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.10855111, 0.28903672, 0.2550475, -0.0276597...","[0.27581334, 0.80696917, 1.0271364, -0.1037090...",1.0,3.0
8450,сероквель,"[0.087633356, 0.93445277, 0.69432694, 0.298702...",0,6,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[-0.04207938, 0.36281538, 0.25962716, 0.010672...","[0.087633356, 0.93445277, 0.69432694, 0.298702...",1.0,2.0
8541,день,"[0.11187396, 1.2210127, -0.88301444, -0.466371...",0,7,0,0,2,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,"[0.11187396, 1.2210127, -0.88301444, -0.466371...","[0.10426845, 0.58591, -0.40001512, -0.31153095...","[0.3036255, 0.7870125, -0.33815098, -0.5249253...",0.0,0.0
11117,500,"[0.34515736, 0.99082905, 1.1647757, 0.13400415...",0,8,0,0,2,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,"[0.0851192921, 0.789108872, 0.923557222, -0.23...","[0.00030258, -0.03585999, -0.01680361, -0.0245...",[0.0],0.0,0.0
11787,капецитабин,"[0.342338204, 1.20741296, 0.623081028, 0.04668...",0,10,0,0,2,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,"[0.342338204, 1.20741296, 0.623081028, 0.04668...","[0.00206052, -0.05985366, 0.06623499, -0.06572...","[-0.0034975, 0.14409654, 0.24720684, -0.020498...",0.0,0.0
11806,иресса,"[0.403851897, 0.462512106, 0.103562765, -0.530...",0,11,0,0,2,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,"[0.235153243, -0.000665825617, 0.172356203, 0....","[0.07661632, 0.08160812, 0.05639331, 0.0203737...","[0.12400526, 0.19563669, 0.21844696, -0.525781...",0.0,0.0


In [353]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [354]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [355]:
df.TOKEN_VECT[710]

array([nan])

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [356]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [357]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

Посмотрим на данные.

In [358]:
df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,0.464646,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0.504478,0.837048,0.599038,0.0,0.0
1,препарат,0.506963,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0.504478,0.837048,0.599038,0.0,0.0
2,рисперидон,0.799171,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,0.889792,0.837048,0.835662,1.0,2.0
3,рисполепт,0.799122,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,0.543908,0.837048,0.835210,1.0,3.0
4,сперидан,0.767995,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,0.585486,0.837048,0.812871,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12995,hifu,0.502863,1,15,1,1,18,NaN,NaN,NaN,NaN,NaN,obj,проходил,NaN,NaN,masc,VERB,0.531924,0.545844,0.684629,0.0,0.0
12996,терапию,0.460462,1,15,1,1,19,inan,accs,femn,NaN,NOUN,obj,проходил,NaN,NaN,masc,VERB,0.531924,0.545844,0.684629,0.0,0.0
12997,и,0.421848,1,15,1,1,20,NaN,NaN,NaN,NaN,CONJ,cc,подскажет,NaN,NaN,NaN,VERB,0.499019,0.545844,0.416419,0.0,0.0
12998,подскажет,0.472722,1,15,1,1,21,NaN,NaN,NaN,3per,VERB,conj,проходил,NaN,NaN,masc,VERB,0.531924,0.545844,0.684629,0.0,0.0


In [359]:
df.describe()

,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
count,13000.000000,13000.000000,13000.000000,13000.000000,13000.000000,13000.000000,13000.000000,13000.000000,13000.000000,13000.000000,13000.000000
mean,0.480690,0.832538,5.289308,14.889692,6.726154,977.064462,0.469394,0.560416,0.567353,0.091154,55.267615
std,0.092710,0.373402,3.315696,14.301254,9.930395,964.237560,0.094269,0.090790,0.142743,0.395274,316.869244
min,-0.043571,0.000000,1.000000,0.000000,0.000000,0.000000,-0.073175,0.335493,-0.027355,0.000000,0.000000
25%,0.415826,1.000000,3.000000,3.000000,1.000000,115.000000,0.420075,0.501624,0.504398,0.000000,0.000000
50%,0.465431,1.000000,5.000000,9.000000,3.000000,707.000000,0.473326,0.550356,0.588802,0.000000,0.000000
75%,0.530893,1.000000,7.000000,23.000000,8.000000,1625.000000,0.521310,0.610899,0.657608,0.000000,0.000000
max,0.833344,1.000000,15.000000,51.000000,60.000000,3668.000000,0.889792,0.860560,0.921520,3.000000,3653.000000


## **Модели. Обучение, оценка.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [360]:
for_factorize = 'ANIMACY	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS']

In [361]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1, -1])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [362]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2', 'voct'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl', 'csubj:pass'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'химиотерапию', 'который', 'инореканом', 'нибудь', 'отзовитесь',
        'методе', 'благодарна', 'инорекане', 'отзывам', 'масловы'],
       dtype='object', length=2366),
 'HEAD_ANIMACY': Index(['inan', 'anim'], d

In [363]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [364]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,0.799171,0,1,0,0,2,1,0,0,-1,1,2,1,0,0,0,1,0.889792,0.837048,0.835662,1.0,2.0
3,рисполепт,0.799122,0,1,0,0,3,0,1,1,-1,1,3,2,1,0,0,1,0.543908,0.837048,0.835210,1.0,3.0
4,сперидан,0.767995,0,1,0,0,4,-1,-1,0,-1,2,4,3,0,1,1,1,0.585486,0.837048,0.812871,1.0,4.0
21,лекарством,0.435640,0,1,1,1,10,0,5,2,-1,1,4,7,-1,-1,-1,3,0.500487,0.509296,0.633306,2.0,0.0
142,рисполепта,0.726008,0,1,1,13,7,0,0,1,-1,1,21,69,0,2,1,1,0.464445,0.625097,0.754986,1.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12726,приходили,0.505865,0,15,0,10,1,-1,-1,-1,-1,0,0,2307,-1,-1,-1,0,0.494902,0.478106,0.606547,1.0,204.0
12785,завершению,0.405717,0,15,0,16,1,0,2,2,-1,1,10,1851,-1,-1,-1,0,0.386212,0.420887,0.452415,1.0,263.0
12854,к,0.380843,0,15,0,22,3,-1,-1,-1,-1,7,9,2332,1,2,1,1,0.610706,0.629229,0.397894,1.0,332.0
12876,стадию,0.504984,0,15,0,25,7,0,3,1,-1,1,1,1777,-1,-1,-1,0,0.456900,0.680371,0.595394,2.0,0.0


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [365]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)

In [366]:
train = train.reset_index()

In [367]:
# train = train.iloc[:,:24].drop(['TOKEN', 'level_0', 'index'], axis=1)
# train = train.drop(['TOKEN', 'index'], axis=1)

In [368]:
test = test.reset_index()

In [369]:
test.head(5)

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,5987,лекарства,0.381482,1,4,7,1,363,0,3,2,-1,1,4,1202,0,0,0,1,0.539367,0.670210,0.921520,2.0,0.0
1,12695,и,0.421848,0,15,0,7,8,-1,-1,-1,-1,4,13,2301,0,5,2,1,0.476938,0.634421,0.541925,3.0,165.0
2,6270,он,0.486446,1,5,3,2,68,-1,0,0,1,5,12,1270,-1,-1,0,13,0.576194,0.619078,0.637224,3.0,41.0
3,7971,его,0.492534,1,5,45,9,1769,-1,3,0,1,5,11,254,0,2,0,1,0.576415,0.597164,0.623159,3.0,1770.0
4,5882,лекарства,0.381482,1,4,6,5,258,0,3,2,-1,1,21,1225,0,0,2,1,0.384466,0.456293,0.540911,2.0,287.0


In [370]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [371]:
# test.loc[test['NER'] != 0]

In [372]:
train.loc[(train['NER'] == 1.0) | (train['NER'] == 2.0 )]

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
1,1662,препарат,0.506963,1,2,18,2,997,0,0,0,-1,1,12,344,-1,-1,-1,0,0.574016,0.682645,0.747317,2.0,1081.0
2,11399,ног,0.449428,0,9,0,25,15,0,1,1,-1,1,21,2083,0,1,0,1,0.359050,0.470542,0.482783,2.0,301.0
4,10079,химию,0.567718,1,7,8,10,1176,0,3,1,-1,1,12,1859,-1,-1,1,0,0.543043,0.429914,0.624145,2.0,0.0
6,366,препарата,0.427066,1,1,2,5,86,0,1,0,-1,1,21,154,0,1,1,1,0.409713,0.541987,0.533997,2.0,344.0
7,7405,нейролептика,0.698133,1,5,36,3,1203,0,1,0,-1,1,21,1456,0,0,2,1,0.473185,0.628176,0.736899,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,8987,хт,0.586058,1,7,1,8,84,-1,-1,-1,-1,-1,21,1654,0,1,2,1,0.518647,0.606840,0.562300,2.0,0.0
226,421,препарат,0.506963,1,1,5,0,141,0,0,0,-1,1,0,1,0,0,0,1,0.889792,0.602804,0.662954,2.0,2.0
227,8667,спазмолитиками,0.546850,0,7,0,10,12,0,5,0,-1,1,10,1668,-1,-1,-1,0,0.620670,0.630372,0.586664,2.0,0.0
229,10037,химию,0.567718,1,7,8,6,1134,0,3,1,-1,1,1,1851,-1,-1,-1,0,0.386212,0.577570,0.659959,2.0,0.0


In [373]:
# ['TOPIC_NUM',	'POST_NUM',	'TOKEN_NUM',	'GENDER',	'PERSON', 'TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 
#  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',	'POS',	'DEPENDENCY',	'HEAD',	'HEAD_POS']

Делаем наборы для обучения моделей.

In [374]:
X, y, X_test, y_test = train.drop(['POST_NUM',	'SENT_NUM', 'TOKEN','IS_ANSWER',  'HEAD_VECT', 'HEAD_CHILDS_VECT', 'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',		'DEPENDENCY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop([ 'POST_NUM', 	'SENT_NUM', 'TOKEN', 'IS_ANSWER',  'HEAD_VECT', 'HEAD_CHILDS_VECT', 'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',		'DEPENDENCY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

In [375]:
# X, y, X_test, y_test = train.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.COREFERENCE_CLUSTER

In [376]:
X

,TOKEN_VECT,TOPIC_NUM,TOKEN_NUM,GENDER,PERSON,POS,SENT_VECT,NER,COREFERENCE_CLUSTER
0,0.403431,5,539,-1,1,5,0.539287,3.0,521.0
1,0.506963,2,997,0,-1,1,0.682645,2.0,1081.0
2,0.449428,9,15,1,-1,1,0.470542,2.0,301.0
3,0.490864,2,160,0,1,5,0.539796,3.0,182.0
4,0.567718,7,1176,1,-1,1,0.429914,2.0,0.0
...,...,...,...,...,...,...,...,...,...
226,0.506963,1,141,0,-1,1,0.602804,2.0,2.0
227,0.546850,7,12,0,-1,1,0.630372,2.0,0.0
228,0.492534,3,2017,0,1,5,0.818542,3.0,2031.0
229,0.567718,7,1134,1,-1,1,0.577570,2.0,0.0


In [377]:
X_test.head(10)

,TOKEN_VECT,TOPIC_NUM,TOKEN_NUM,GENDER,PERSON,POS,SENT_VECT,NER,COREFERENCE_CLUSTER
0,0.381482,4,363,2,-1,1,0.670210,2.0,0.0
1,0.421848,15,8,-1,-1,4,0.634421,3.0,165.0
2,0.486446,5,68,0,1,5,0.619078,3.0,41.0
3,0.492534,5,1769,0,1,5,0.597164,3.0,1770.0
4,0.381482,4,258,2,-1,1,0.456293,2.0,287.0
5,0.427066,5,2127,0,-1,1,0.454711,2.0,0.0
6,0.408908,11,21,-1,-1,7,0.537527,2.0,133.0
7,0.444235,3,3618,1,1,5,0.486946,3.0,3619.0
8,0.396613,4,368,-1,1,5,0.799615,3.0,0.0
9,0.486446,3,3255,0,1,5,0.671927,3.0,3270.0


In [378]:
y

0       521.0
1      1081.0
2       301.0
3       182.0
4         0.0
        ...  
226       2.0
227       0.0
228    2031.0
229       0.0
230    1473.0
Name: COREFERENCE_CLUSTER, Length: 231, dtype: float64

Обучаем модели.

In [379]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test), 

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2000, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [380]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=2000, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [381]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, min_samples_split=2, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.4827586206896552

In [382]:
regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)

1.0

In [331]:
from sklearn import svm

In [384]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.3275862068965517

In [386]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='kd_tree', leaf_size=1000, p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.6379310344827587

In [387]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


Функция для оценки моделей

In [388]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [389]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [390]:
# my_eval(list(naive_pred), list(y_test))

In [391]:
my_eval(list(pred_knn), list(y_test))

18  true positive
18  false positive
19  true negative
3  false negative
---------
0.5  presicion
0.8571428571428571  recall
0.631578947368421  f-score


In [392]:
my_eval(list(pred), list(y_test))

15  true positive
24  false positive
19  true negative
0  false negative
---------
0.38461538461538464  presicion
1.0  recall
0.5555555555555556  f-score


In [393]:
my_eval(list(pred_randf), list(y_test))

9  true positive
30  false positive
19  true negative
0  false negative
---------
0.23076923076923078  presicion
1.0  recall
0.375  f-score


In [394]:
my_eval(list(pred_entropy), list(y_test))

17  true positive
22  false positive
19  true negative
0  false negative
---------
0.4358974358974359  presicion
1.0  recall
0.6071428571428571  f-score


In [395]:
my_eval(list(pred_regr), list(y_test))

0  true positive
58  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [396]:
my_eval(list(pred_svm), list(y_test))

0  true positive
15  false positive
19  true negative
24  false negative
---------
0.0  presicion
0.0  recall
0  f-score
